In [1]:
!cd /kaggle/working/
!rm -rf *

In [2]:
!pip install split-folders

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, models, transforms
import os
import shutil
import time
import cv2
import pandas as pd
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, ConcatDataset
from PIL import Image
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import splitfolders

In [4]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
input_path = "/kaggle/input/realwaste-dataset/realwaste-main/RealWaste"

working_path = "/kaggle/working/realwaste"

shutil.copytree(input_path, working_path, dirs_exist_ok=True)

print("Dataset copied successfully to working directory!")

Dataset copied successfully to working directory!


In [6]:
base_dir = "/kaggle/working/realwaste"
for folder in os.listdir(base_dir):
    count = len(os.listdir(os.path.join(base_dir, folder)))
    print(f"{folder}: {count} images")

Miscellaneous Trash: 495 images
Food Organics: 411 images
Plastic: 921 images
Cardboard: 461 images
Textile Trash: 318 images
Glass: 420 images
Vegetation: 436 images
Paper: 500 images
Metal: 790 images


In [7]:
# Configuration
IMG_SIZE = (224, 224)
DATA_DIR = "/kaggle/input/realwaste/realwaste-main/RealWaste"

In [8]:
input_folder = base_dir
split_dir = "/kaggle/working/RealWaste_split"

splitfolders.ratio(input_folder, output=split_dir, seed=42, ratio=(.7, .15, .15))

Copying files: 4752 files [00:01, 4415.63 files/s]


In [9]:
# Count images in each subfolder
for split in ['train', 'val', 'test']:
    split_path = os.path.join(split_dir, split)
    print(f"\n{split.upper()} SET")
    total = 0
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        count = len(os.listdir(cls_path))

        total += count
        print(f"  {cls}: {count} images")
    print(f" Total {split}: {total} images")


TRAIN SET
  Miscellaneous Trash: 346 images
  Food Organics: 287 images
  Plastic: 644 images
  Cardboard: 322 images
  Textile Trash: 222 images
  Glass: 294 images
  Vegetation: 305 images
  Paper: 350 images
  Metal: 553 images
 Total train: 3323 images

VAL SET
  Miscellaneous Trash: 74 images
  Food Organics: 61 images
  Plastic: 138 images
  Cardboard: 69 images
  Textile Trash: 47 images
  Glass: 63 images
  Vegetation: 65 images
  Paper: 75 images
  Metal: 118 images
 Total val: 710 images

TEST SET
  Miscellaneous Trash: 75 images
  Food Organics: 63 images
  Plastic: 139 images
  Cardboard: 70 images
  Textile Trash: 49 images
  Glass: 63 images
  Vegetation: 66 images
  Paper: 75 images
  Metal: 119 images
 Total test: 719 images


In [10]:
# Data Augmentation
train_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=45),
        transforms.RandomResizedCrop(size=IMG_SIZE[0], scale=(0.6, 0.9)),
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor()
])

val_test_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder("/kaggle/working/RealWaste_split/train", transform=train_transforms)
val_dataset   = datasets.ImageFolder("/kaggle/working/RealWaste_split/val", transform=val_test_transforms)
test_dataset  = datasets.ImageFolder("/kaggle/working/RealWaste_split/test", transform=val_test_transforms)

# Get number of classes and device
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")

Number of classes: 9


In [11]:
# Increase the number of training data
duplication_factor = 3
train_dataset_augmented = ConcatDataset([train_dataset] * duplication_factor)

In [12]:
# Extract targets
targets = []
for dataset in train_dataset_augmented.datasets:  # train_dataset_augmented.datasets is a list
    if hasattr(dataset, 'targets'):
        targets.extend(dataset.targets)
    else:
        # fallback if custom dataset
        for i in range(len(dataset)):
            _, label = dataset[i]
            targets.append(label)

# Convert to numpy
targets = np.array(targets)

# Class counts
class_counts = np.bincount(targets)
print("Class counts:", class_counts)

base_dataset = train_dataset_augmented.datasets[0]  # first dataset in the concat list
classes = base_dataset.classes

# Compute class weights (inverse of frequency)
class_weights = 1. / class_counts

# Ensure sample_weights are floats for the sampler
sample_weights = [class_weights[label].item() for label in targets]

print("Class counts per category:")
for cls, count in zip(classes, class_counts):
    print(f"  {cls:15s}: {count}")

print("\n Class weights (inverse of frequency):")
for cls, w in zip(classes, class_weights):
    print(f"  {cls:15s}: {w:.6f}")

Class counts: [ 966  861  882 1659 1038 1050 1932  666  915]
Class counts per category:
  Cardboard      : 966
  Food Organics  : 861
  Glass          : 882
  Metal          : 1659
  Miscellaneous Trash: 1038
  Paper          : 1050
  Plastic        : 1932
  Textile Trash  : 666
  Vegetation     : 915

 Class weights (inverse of frequency):
  Cardboard      : 0.001035
  Food Organics  : 0.001161
  Glass          : 0.001134
  Metal          : 0.000603
  Miscellaneous Trash: 0.000963
  Paper          : 0.000952
  Plastic        : 0.000518
  Textile Trash  : 0.001502
  Vegetation     : 0.001093


In [13]:
# Define a consistent BATCH_SIZE
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset_augmented, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False) 
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


print(f"\n WeightedRandomSampler created successfully!")
print(f" Total samples in epoch: {len(sample_weights)}")
print(f" Training Batch size: {train_loader.batch_size}")
print(f" Validation Batch size: {val_loader.batch_size}")
print(f" Total training batches per epoch: {len(train_loader)}")


 WeightedRandomSampler created successfully!
 Total samples in epoch: 9969
 Training Batch size: 32
 Validation Batch size: 32
 Total training batches per epoch: 312


In [14]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []


    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0
    
        # TRAINING LOOP
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Train)", leave=False)
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item() * images.size(0)
    
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_acc = 100 * correct / total
            train_loop.set_postfix(loss=loss.item(), acc=train_acc)
    
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * correct / total

        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        # VALIDATION LOOP
        model.eval()
        val_running_loss = 0
        val_correct = 0
        val_total = 0
    
        with torch.no_grad():
            val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Val)", leave=False)
            for images, labels in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
    
                val_running_loss += loss.item() * images.size(0)
    
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
    
        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * val_correct / val_total

        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)
        
        # Summary
        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}% | "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")

    return train_losses, val_losses, train_accuracies, val_accuracies

In [15]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=9):
        super(CNNModel, self).__init__()

        # Convolution Blocks
        self.conv_block1 = self._create_conv_block(3, 64)  # 64 filters
        self.conv_block2 = self._create_conv_block(64, 128)  # 128 filters
        self.conv_block3 = self._create_conv_block(128, 256)  # 256 filters
        self.conv_block4 = self._create_conv_block(256, 512)  # 512 filters
        self.conv_block5 = self._create_conv_block(512, 512)  # 512 filters
        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Fully Connected Layers
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.3)

    def _create_conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)

        # Global Average Pooling
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten

        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

model = CNNModel(num_classes=num_classes)
model_name = "Custom CNN"

# Loss Function
weights = torch.tensor(class_weights.copy(), dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

model = model.to(device)

results = {}

def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

In [16]:
print(f"\n===== Training with Adam =====\n")

reset_weights(model)

# Adams Optimizer 
learning_rate = 1e-5
weight_decay = 1e-2
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Scheduler of learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

num_epochs = 20

# Training the model
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs
)

# Save the model
results['Adam'] = (train_losses, val_losses, train_accuracies, val_accuracies)

model_save_path = f"/kaggle/working/trained_{model_name}_adam.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


===== Training with Adam =====



Epoch 1/20: Train Loss: 1.5036, Train Acc: 48.86% | Val Loss: 1.3147, Val Acc: 54.23%


Epoch 2/20: Train Loss: 1.1043, Train Acc: 61.71% | Val Loss: 1.0890, Val Acc: 59.44%


Epoch 3/20: Train Loss: 0.9315, Train Acc: 67.44% | Val Loss: 0.9674, Val Acc: 66.20%


Epoch 4/20: Train Loss: 0.8444, Train Acc: 69.89% | Val Loss: 0.8754, Val Acc: 68.87%


...

Epoch 16/20: Train Loss: 0.3914, Train Acc: 85.72% | Val Loss: 0.6076, Val Acc: 78.31%


Epoch 17/20: Train Loss: 0.3638, Train Acc: 86.57% | Val Loss: 0.5865, Val Acc: 78.73%


Epoch 18/20: Train Loss: 0.3688, Train Acc: 86.57% | Val Loss: 0.5542, Val Acc: 79.58%


Epoch 19/20: Train Loss: 0.3306, Train Acc: 88.14% | Val Loss: 0.5004, Val Acc: 83.10%


Epoch 20/20: Train Loss: 0.3354, Train Acc: 87.80% | Val Loss: 0.5163, Val Acc: 81.55%
Model saved to /kaggle/working/trained_Custom CNN_adam.pth


In [ ]:
print(f"\n===== Training with SGD =====\n")

reset_weights(model)

# SGD Optimizer 
learning_rate = 1e-3
weight_decay = 1e-4
optimizer = optim.SGD(
    model.parameters(), 
    lr= learning_rate, 
    weight_decay= weight_decay
)

# Scheduler of learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

num_epochs = 20

# Training the model
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs
)

# Save the model
results['SGD'] = (train_losses, val_losses, train_accuracies, val_accuracies)

model_save_path = f"/kaggle/working/trained_{model_name}_sgd.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


===== Training with SGD =====



Epoch 1/20: Train Loss: 1.4416, Train Acc: 67.45% | Val Loss: 1.1180, Val Acc: 69.15%


Epoch 2/20: Train Loss: 0.8069, Train Acc: 81.43% | Val Loss: 1.1299, Val Acc: 58.45%


Epoch 3/20: Train Loss: 0.6074, Train Acc: 83.86% | Val Loss: 1.2246, Val Acc: 54.79%


Epoch 4/20: Train Loss: 0.5193, Train Acc: 84.75% | Val Loss: 0.8920, Val Acc: 68.03%


...

Epoch 11/20: Train Loss: 0.3084, Train Acc: 89.50% | Val Loss: 0.5920, Val Acc: 78.03%


Epoch 12/20: Train Loss: 0.3125, Train Acc: 88.94% | Val Loss: 1.4849, Val Acc: 57.61%


Epoch 13/20: Train Loss: 0.2939, Train Acc: 89.84% | Val Loss: 0.5879, Val Acc: 80.42%


Epoch 14/20: Train Loss: 0.2845, Train Acc: 89.98% | Val Loss: 0.8045, Val Acc: 71.41%


Epoch 15/20 (Train):   6%|▌         | 18/312 [00:07<02:02,  2.39it/s, acc=90.8, loss=0.211] 

In [ ]:
print(f"\n===== Training with SGD with Momentum =====\n")

reset_weights(model)

# SGD with Momentum Optimizer 
learning_rate = 1e-4
weight_decay = 1e-4
optimizer = optim.SGD(
    model.parameters(), 
    lr= learning_rate, 
    weight_decay= weight_decay,
    momentum=0.8
)

# Scheduler of learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

num_epochs = 20

# Training the model
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs
)

# Save the model
results['SGD with Momentum'] = (train_losses, val_losses, train_accuracies, val_accuracies)

model_save_path = f"/kaggle/working/trained_{model_name}_sgdmomentum.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
plt.figure(figsize=(14, 10))

# 1️ Train Loss
plt.subplot(2, 2, 1)
for name in results:
    plt.plot(results[name][0], label=f"{name}")
plt.title("Training Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# 2️ Validation Loss
plt.subplot(2, 2, 2)
for name in results:
    plt.plot(results[name][1], label=f"{name}")
plt.title("Validation Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# 3️ Training Accuracy
plt.subplot(2, 2, 3)
for name in results:
    plt.plot(results[name][2], label=f"{name}")
plt.title("Training Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid(True)

# 4️ Validation Accuracy
plt.subplot(2, 2, 4)
for name in results:
    plt.plot(results[name][3], label=f"{name}")
plt.title("Validation Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
print("\n===== Validation Accuracy Comparison =====\n")
for name in results:
    val_acc_list = results[name][3]   
    if len(val_acc_list) > 0:
        final_val_acc = val_acc_list[-1]  
        print(f"{name} Final Validation Accuracy: {final_val_acc:.2f}%")
    else:
        print(f"{name} has no validation records. (Check training loop)")

print("\n===== Best Validation Accuracy for Each Optimizer =====\n")
for name in results:
    val_acc_list = results[name][3]
    if len(val_acc_list) > 0:
        best_val_acc = max(val_acc_list)
        print(f"{name} Best Validation Accuracy: {best_val_acc:.2f}%")
    else:
        print(f"{name} has no validation records.")

In [ ]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = 100 * correct / total

    return avg_test_loss, test_accuracy

In [ ]:
model_paths = {
    "Adam": "/kaggle/working/trained_Custom CNN_adam.pth",
    "SGD": "/kaggle/working/trained_Custom CNN_sgd.pth",
    "SGD with Momentum": "/kaggle/working/trained_Custom CNN_sgdmomentum.pth"
}

print("\n===== Test Set Evaluation =====\n")

for name, path in model_paths.items():
    # Load fresh model architecture
    test_model = CNNModel(num_classes=9).to(device)
    test_model.load_state_dict(torch.load(path, map_location=device))

    test_loss, test_acc = evaluate_model(test_model, test_loader, criterion)

    print(f"{name} Optimizer -> Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")


In [ ]:
def evaluate_detailed(model, loader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
                xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
model_paths = {
    "Adam": "/kaggle/working/trained_Custom CNN_adam.pth",
    "SGD": "/kaggle/working/trained_Custom CNN_sgd.pth",
    "SGD with Momentum": "/kaggle/working/trained_Custom CNN_sgdmomentum.pth"
}

print("\n===== Detailed Test Evaluation for Each Optimizer =====\n")

for name, path in model_paths.items():
    print(f"\n--- {name} Optimizer ---\n")

    test_model = CNNModel(num_classes=9).to(device)
    test_model.load_state_dict(torch.load(path, map_location=device))

    evaluate_detailed(test_model, test_loader, classes)


Pre-trained Models

In [ ]:
# Data Augmentation

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    x: datasets.ImageFolder(f"{split_dir}/{x}", data_transforms[x])
    for x in ["train", "val"]
}
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
    for x in ["train", "val"]
}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val"]}
class_names = image_datasets["train"].classes

In [ ]:
# Training Function (with Early Stopping)
def train_model(model, criterion, optimizer, scheduler, num_epochs, patience=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stop_counter = 0

    train_losses, val_losses, train_accs, val_accs = [], [], [], []

    print(f"\n========== Training {model.__class__.__name__} ==========\n")

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}\n" + "-"*30)

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == "train":
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc.item())

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f}, {phase.capitalize()} Acc: {epoch_acc:.4f}")

            # Early stopping based on validation accuracy
            if phase == "val":
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1

        print()
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best Val Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model, train_losses, val_losses, train_accs, val_accs

In [ ]:
# Model Definition Function

def get_model(model_name, num_classes, fine_tune=True, dropout_p=0.5):
    if model_name == "resnet":
        model = models.resnet18(pretrained=True)
        if not fine_tune:
            for param in model.parameters():
                param.requires_grad = False
        in_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Dropout(dropout_p),
            nn.Linear(in_features, num_classes)
        )

    elif model_name == "vgg":
        model = models.vgg16(pretrained=True)
        if not fine_tune:
            for param in model.features.parameters():
                param.requires_grad = False
        in_features = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
            nn.Dropout(dropout_p),
            nn.Linear(in_features, num_classes)
        )

    return model.to(DEVICE)

In [ ]:
NUM_EPOCHS_COMP = 20 
criterion = nn.CrossEntropyLoss()
LEARNING_RATE = 0.001
num_classes = len(class_names)

# --- 1. ResNet-18 Transfer Learning ---

model_resnet = get_model("resnet", num_classes)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler_resnet = StepLR(optimizer_resnet, step_size=10, gamma=0.1)
model_resnet, resnet_train_loss, resnet_val_loss, resnet_train_acc, resnet_val_acc = train_model(
    model_resnet, criterion, optimizer_resnet, scheduler_resnet, num_epochs=NUM_EPOCHS_COMP
)

# --- 2. VGG-16 Transfer Learning ---

model_vgg = get_model("vgg", num_classes)
optimizer_vgg = optim.Adam(model_vgg.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler_vgg = StepLR(optimizer_vgg, step_size=10, gamma=0.1)
model_vgg, vgg_train_loss, vgg_val_loss, vgg_train_acc, vgg_val_acc = train_model(
    model_vgg, criterion, optimizer_vgg, scheduler_vgg, num_epochs=NUM_EPOCHS_COMP
)

In [ ]:
# Plotting Functions
def plot_metrics(train_loss, val_loss, train_acc, val_acc, title):
    epochs = range(1, len(train_loss) + 1)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label="Train Loss")
    plt.plot(epochs, val_loss, label="Val Loss")
    plt.title(f"{title} - Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label="Train Acc")
    plt.plot(epochs, val_acc, label="Val Acc")
    plt.title(f"{title} - Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

plot_metrics(resnet_train_loss, resnet_val_loss, resnet_train_acc, resnet_val_acc, "ResNet18")
plot_metrics(vgg_train_loss, vgg_val_loss, vgg_train_acc, vgg_val_acc, "VGG16")

In [ ]:
# Test data transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load test dataset
test_dataset = datasets.ImageFolder('/kaggle/working/realwaste_split/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
#For ResNet18
model_to_test = model_resnet  
model_to_test.eval()          
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_to_test(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate test accuracy
test_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f" Test Accuracy: {test_accuracy * 100:.2f}%")

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
class_names = test_dataset.classes

#  Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))

#  Overall Metrics
precision_macro = precision_score(all_labels, all_preds, average='macro')
recall_macro = recall_score(all_labels, all_preds, average='macro')
f1_macro = f1_score(all_labels, all_preds, average='macro')

precision_weighted = precision_score(all_labels, all_preds, average='weighted')
recall_weighted = recall_score(all_labels, all_preds, average='weighted')
f1_weighted = f1_score(all_labels, all_preds, average='weighted')

print(f"\n Overall Metrics:")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall:    {recall_macro:.4f}")
print(f"Macro F1-Score:  {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall:    {recall_weighted:.4f}")
print(f"Weighted F1-Score:  {f1_weighted:.4f}")

In [ ]:
#For VGG16
model_to_test = model_vgg  
model_to_test.eval()          

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_to_test(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate test accuracy
test_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f" Test Accuracy: {test_accuracy * 100:.2f}%")

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
class_names = test_dataset.classes

#  Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

#  Classification Report
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))

#  Overall Metrics
precision_macro = precision_score(all_labels, all_preds, average='macro')
recall_macro = recall_score(all_labels, all_preds, average='macro')
f1_macro = f1_score(all_labels, all_preds, average='macro')

precision_weighted = precision_score(all_labels, all_preds, average='weighted')
recall_weighted = recall_score(all_labels, all_preds, average='weighted')
f1_weighted = f1_score(all_labels, all_preds, average='weighted')

print(f"\n Overall Metrics:")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall:    {recall_macro:.4f}")
print(f"Macro F1-Score:  {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall:    {recall_weighted:.4f}")
print(f"Weighted F1-Score:  {f1_weighted:.4f}")